In [ ]:
# | default_exp task

# Task

> Tasks to interact with the Archivematica REST API


In [ ]:
# | hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# | export
from archivematica_tools.api import ArchivematicaAPIClient
from archivematica_tools.aws import AwsClient
from typing import Optional

In [ ]:
# | export
class Task:

    def __init__(self, dashboard_url: str, dashboard_username: str, dashboard_api_key: str,
                 storage_service_url: str, storage_service_username: str, storage_service_password: str,
                 aws_access_key_id: str, aws_secret_access_key: str):
        """
        Task クラスのコンストラクタ。Matica APIとAWS S3クライアントの初期化を行う。

        Args:
            dashboard_url (str): MaticaダッシュボードのURL。
            dashboard_username (str): Maticaダッシュボードのユーザー名。
            dashboard_api_key (str): MaticaダッシュボードのAPIキー。
            storage_service_url (str): ストレージサービスのURL。
            storage_service_username (str): ストレージサービスのユーザー名。
            storage_service_password (str): ストレージサービスのパスワード。
            aws_access_key_id (str): AWSアクセスキーID。
            aws_secret_access_key (str): AWSシークレットアクセスキー。
        """
        self.matica_client = ArchivematicaAPIClient(dashboard_url, dashboard_username, dashboard_api_key,
                                             storage_service_url, storage_service_username, storage_service_password)
        self.aws_client = AwsClient(aws_access_key_id, aws_secret_access_key, "us-east-1")

    @staticmethod
    def ensure_slash_suffix(s: str) -> str:
        """指定された文字列の末尾にスラッシュがなければ追加する

        Args:
            s (str): 処理する文字列

        Returns:
            str: 末尾にスラッシュが保証された文字列
        """
        return s if s.endswith('/') else f'{s}/'

    @staticmethod
    def main(dashboard_url: str, dashboard_username: str, dashboard_api_key: str,
             storage_service_url: str, storage_service_username: str, storage_service_password: str,
             aws_access_key_id: str, aws_secret_access_key: str, task_id: str, transfer_type: str, 
             transfer_name: str, file_path: str, location_uuid: str, processing_config: str, 
             bucket_name: str, transfer_source_prefix: str) -> Optional[str]:
        """
        メインの処理を実行する静的メソッド。ZIPファイルをS3にアップロードし、Maticaでの処理をトリガーする。

        Args:
            以下の引数は __init__ メソッドと同様。
            task_id (str): タスクID。
            transfer_type (str): 転送タイプ。
            transfer_name (str): 転送名。
            file_path (str): ファイルパス。
            location_uuid (str): 位置UUID。
            processing_config (str): 処理設定。
            bucket_name (str): バケット名。
            transfer_source_prefix (str): 転送ソースプレフィックス。

        Returns:
            Optional[str]: 成功時には処理されたファイルのURLを返し、失敗時にはNoneを返す。
        """
        
        task = Task(dashboard_url, 
        dashboard_username,
        dashboard_api_key,
        storage_service_url,
        storage_service_username,
        storage_service_password,
        aws_access_key_id,
        aws_secret_access_key)

        matica_client = task.matica_client

        transfer_source_prefix = Task.ensure_slash_suffix(transfer_source_prefix)
        s3_path = f"{transfer_source_prefix}{task_id}"

        # bucket_name = "archivematica.aws.ldas.jp"
        # bucket_name

        task.aws_client.upload_to_s3_zip(file_path, s3_path, bucket_name)

        # location_uuid = "8ba875f0-0e78-4c98-a608-751a362e3194"
        # processing_config = "automated"

        transfer_UUID = task.matica_client.v2beta_package(transfer_type, "", location_uuid, "/" + s3_path, transfer_name, processing_config)

        sip_UUID = matica_client.check_transfer_status(transfer_UUID)
        ingest_UUID = matica_client.ingest(sip_UUID)

        url = "https://s3.amazonaws.com/" + bucket_name + matica_client.get_current_full_path(ingest_UUID)

        return url

In [ ]:
show_doc(Task)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/task.py#L13){target="_blank" style="float:right; font-size:smaller"}

### Task

>      Task (dashboard_url:str, dashboard_username:str, dashboard_api_key:str,
>            storage_service_url:str, storage_service_username:str,
>            storage_service_password:str, aws_access_key_id:str,
>            aws_secret_access_key:str)

Task クラスのコンストラクタ。Matica APIとAWS S3クライアントの初期化を行う。

Args:
    dashboard_url (str): MaticaダッシュボードのURL。
    dashboard_username (str): Maticaダッシュボードのユーザー名。
    dashboard_api_key (str): MaticaダッシュボードのAPIキー。
    storage_service_url (str): ストレージサービスのURL。
    storage_service_username (str): ストレージサービスのユーザー名。
    storage_service_password (str): ストレージサービスのパスワード。
    aws_access_key_id (str): AWSアクセスキーID。
    aws_secret_access_key (str): AWSシークレットアクセスキー。

# Main

In [ ]:
show_doc(Task.main)

---

[source](https://github.com/nakamura196/matica_tools/blob/main/matica_tools/task.py#L30){target="_blank" style="float:right; font-size:smaller"}

### Task.main

>      Task.main (dashboard_url:str, dashboard_username:str,
>                 dashboard_api_key:str, storage_service_url:str,
>                 storage_service_username:str, storage_service_password:str,
>                 aws_access_key_id:str, aws_secret_access_key:str, task_id:str,
>                 transfer_type:str, transfer_name:str, file_path:str,
>                 location_uuid:str, processing_config:str, bucket_name:str,
>                 transfer_source_prefix:str)

メインの処理を実行する静的メソッド。ZIPファイルをS3にアップロードし、Maticaでの処理をトリガーする。

Args:
    以下の引数は __init__ メソッドと同様。
    task_id (str): タスクID。
    transfer_type (str): 転送タイプ。
    transfer_name (str): 転送名。
    file_path (str): ファイルパス。
    location_uuid (str): 位置UUID。
    processing_config (str): 処理設定。
    bucket_name (str): バケット名。
    transfer_source_prefix (str): 転送ソースプレフィックス。

Returns:
    Optional[str]: 成功時には処理されたファイルのURLを返し、失敗時にはNoneを返す。

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()